# Práctica 9
# Perceptrón simple y Adaline

# Ejercicio 2

# Santiago Blasco Arnaiz

In [1]:
from sklearn.linear_model import SGDRegressor
from sklearn.datasets import load_boston
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

### Cargamos el dataset

In [2]:
boston = load_boston()
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Como podemos ver en la descripción del dataset este consta de 506 muestras y 12 atributos numéricos continuos, 1 atributo categórico codificado de forma binaria y la clase, que también es númerica continua y representa el valor medio de las casas ocupadas por sus dueños en miles de dolares. 

### Normalizamos los datos

In [3]:
scaler = MinMaxScaler()

X = scaler.fit_transform(boston.data)
Y = boston.target
X.shape,Y.shape

((506, 13), (506,))

### Adaline

In [4]:
#Creamos el modelo Adaline
reg = SGDRegressor(tol = 1e-6, max_iter = 5000, random_state = 0)

In [5]:
#Número de carpetas
K = 10

#Generamos las particiones para cross validation
skf = KFold(n_splits = K)

#Creamos un dataframe para introducir toda la información y mostrarla
r_reg = pd.DataFrame(columns = ['Fold','R2','Aciertos'])

#Contador para variar la carpeta
k = 0

#Iterador de las carpetas de Cross Validation
for train_index, test_index in skf.split(X,Y):
    #Entrenamos con los datos de las carpetas correspondientes
    reg.fit(X[train_index], Y[train_index])
    #Predecimos con los datos de la carpeta correspondiente
    Y_predict = reg.predict(X[test_index])

    #Calculamos el error relativo
    relative_error = abs(Y[test_index] - Y_predict) / Y[test_index]
    
    #Porcentaje de muestras con un error relativo menor que 0.15
    acc = len(np.where(relative_error < 0.15)[0]) / test_index.shape[0]

    #Añadimos los datos obtenidos al dataframe de la solución
    r_reg.loc[k,:] = [k,reg.score(X[test_index], Y[test_index]), acc]
    
    k += 1
    
r_reg

,Fold,R2,Aciertos
0,0,0.766476,0.705882
1,1,0.644545,0.764706
2,2,-0.171073,0.72549
3,3,0.63155,0.568627
4,4,0.596121,0.588235
5,5,0.743927,0.72549
6,6,0.465113,0.7
7,7,0.084458,0.24
8,8,-0.471057,0.32
9,9,0.420804,0.62


### Media y desviación estándar de R2

In [6]:
print("Media = ", r_reg.mean().values[1], "\nDesviación estándar = ", r_reg.std().values[1])

Media =  0.37108645399397555 
Desviación estándar =  0.41968452076301066


No podemos decir mucho respecto a la bondad del ajuste del modelo de la clase, ya que a parte de obtener unos valores bajos (media inferior al 0.4) este coeficiente no es del todo fiable por tener en cuenta de igual forma los atributos con poca relación, además la desviación estándar es bastante alta por lo que hay mucha diferencia entre los valores.


### Media y desviación estándar de la tasa de acierto

In [7]:
print("Media = ", r_reg.mean().values[2], "\nDesviación estándar = ", r_reg.std().values[2])

Media =  0.595843137254902 
Desviación estándar =  0.17925293396864206


La media de las tasas de acierto no es muy alta, además teniendo en cuenta que el valor límite utilizado para el error relativo es de un 15% indica que estos resultados son aún peores ya que son unas tasas de acierto bastante bajas para una tolerancia tan alta.